In [1]:
#Transform Dependencies
import pandas as pd
import numpy as np
import datetime as dt
import dateutil.parser
import re
import json
import os
import plotly.plotly as py
# import cufflinks as cf
from pprint import pprint

#Visualizations
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, Table, MetaData

# EXTRACT

In [2]:
#Load in file scraped from https://planet4589.org/space/log/launchlogy.txt as text and converted to CSV
launchlog = pd.read_csv('Resources/launchlogall.csv')


#Load in files downloaded as CSV from kaggle.com

spacexdf = pd.read_csv('.\\Resources\\spaceX.csv')
nasadf = pd.read_csv('.\\Resources\\NASA_Facilities.csv')

#Load in file saved as JSON, retrieved from https://api.spacexdata.com/v2/launches
spacex = pd.read_json('.\\Resources\\spaceX.json')

#Load in satellites CSV
input_file = ".\\Resources\\UCS_Satellite_Database_12.1.18_CSV.csv"
UCSSatellite_db = pd.read_csv(input_file, encoding="cp1252")

# Transform

Launch Log

In [3]:
launchlog.columns

Index(['Launch', 'Launch Date (UTC)', 'COSPAR', 'PL Name', 'Orig PL Name',
       'SATCAT', 'LV Type', 'LV S/N', 'Site', 'Suc', 'Ref'],
      dtype='object')

In [4]:
launchlog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8641 entries, 0 to 8640
Data columns (total 11 columns):
Launch               5464 non-null object
Launch Date (UTC)    5464 non-null object
COSPAR               8641 non-null object
PL Name              8641 non-null object
Orig PL Name         8641 non-null object
SATCAT               8641 non-null object
LV Type              5464 non-null object
LV S/N               5464 non-null object
Site                 5464 non-null object
Suc                  5464 non-null object
Ref                  5464 non-null object
dtypes: object(11)
memory usage: 742.7+ KB


In [5]:
launchlog = launchlog.rename(index=str, columns={"Launch": "launch", 
                                     "Launch Date (UTC)": "launch_date_utc",
                                     "COSPAR": "COSPAR",
                                     "PL Name": "pl_name",
                                     "Orig PL Name": "orig_pl_name",
                                     "SATCAT": "SATCAT",
                                     "LV Type": "lv_type",
                                     "LV S/N": "lv_sn",
                                     "Site": "site",
                                     "Suc": "suc"                                     
                                    })
launchlog.columns

Index(['launch', 'launch_date_utc', 'COSPAR', 'pl_name', 'orig_pl_name',
       'SATCAT', 'lv_type', 'lv_sn', 'site', 'suc', 'Ref'],
      dtype='object')

In [6]:
launchlog['launch_date_utc'] = launchlog['launch_date_utc'].fillna(method='ffill')
launchlog['launch'] = launchlog['launch'].fillna(method='ffill')

In [7]:
launchlog['launch_date_utc'] = launchlog['launch_date_utc'].fillna(method='ffill')
launchlog['launch_date'] = launchlog['launch_date_utc'].map(lambda x: dateutil.parser.parse(x[0:10]) if pd.notnull(x) else np.nan)\
                                                       .map(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else np.nan)

# Space X

In [8]:
#Parse JSON to get values
spacex['launch_date'] = spacex['launch_date_local'].map(lambda x: dateutil.parser.parse(x[0:10]) if pd.notnull(x) else np.nan)\
                                                       .map(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else np.nan)

spacex['launch_site'] = spacex['launch_site'].apply(lambda x: x.get('site_name_long'))

spacex['rocket_reused'] = spacex['rocket'].apply(lambda x: x.get('fairings') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('reused') if pd.notnull(x) else np.nan)

spacex['rocket_name'] = spacex['rocket'].apply(lambda x: x.get('rocket_name') if pd.notnull(x) else np.nan)

spacex['rocket_type'] = spacex['rocket'].apply(lambda x: x.get('rocket_type') if pd.notnull(x) else np.nan)

spacex['failure_reason'] = spacex['launch_failure_details'].apply(lambda x: x.get('reason') if pd.notnull(x) else np.nan)

spacex['customer'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('customers') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])

spacex['customer_country'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('nationality') if pd.notnull(x) else np.nan)


spacex['orbit'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('orbit') if pd.notnull(x) else np.nan)


spacex['payload_id'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('payload_id') if pd.notnull(x) else np.nan)

spacex['payload_mass_kg'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('payload_mass_kg') if pd.notnull(x) else np.nan)

spacex['payload_type'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('payload_type') if pd.notnull(x) else np.nan)
# Drop unneeded columns
spacex.drop(['details', 'flight_number', 'is_tentative', 'launch_date_local','launch_date_unix', 'launch_date_utc',\
             'launch_failure_details','launch_window', 'launch_year','links', 'mission_id', 'reuse', 'rocket', 'ships',\
             'static_fire_date_unix', 'static_fire_date_utc', 'tbd', 'telemetry','tentative_max_precision', 'timeline',\
             'upcoming'],axis=1,inplace=True)

In [9]:
spacex.head()

,launch_site,launch_success,mission_name,launch_date,rocket_reused,rocket_name,rocket_type,failure_reason,customer,customer_country,orbit,payload_id,payload_mass_kg,payload_type
0,Kwajalein Atoll Omelek Island,False,FalconSat,2006-03-25,False,Falcon 1,Merlin A,merlin engine failure,DARPA,United States,LEO,FalconSAT-2,20.0,Satellite
1,Kwajalein Atoll Omelek Island,False,DemoSat,2007-03-21,False,Falcon 1,Merlin A,harmonic oscillation leading to premature engi...,DARPA,United States,LEO,DemoSAT,NaN,Satellite
2,Kwajalein Atoll Omelek Island,False,Trailblazer,2008-08-02,False,Falcon 1,Merlin C,residual stage-1 thrust led to collision betwe...,NASA,United States,LEO,Trailblazer,NaN,Satellite
3,Kwajalein Atoll Omelek Island,True,RatSat,2008-09-28,False,Falcon 1,Merlin C,NaN,SpaceX,United States,LEO,RatSat,165.0,Satellite
4,Kwajalein Atoll Omelek Island,True,RazakSat,2009-07-13,False,Falcon 1,Merlin C,NaN,ATSB,Malaysia,LEO,RazakSAT,200.0,Satellite


In [10]:
spacexdf.head()

,Flight Number,Launch Date,Launch Time,Launch Site,Vehicle Type,Payload Name,Payload Type,Payload Mass (kg),Payload Orbit,Customer Name,Customer Type,Customer Country,Mission Outcome,Failure Reason,Landing Type,Landing Outcome
0,F1-1,24 March 2006,22:30,Marshall Islands,Falcon 1,FalconSAT-2,Research Satellite,19.5,NaN,DARPA,Government,United States,Failure,Engine Fire During Launch,NaN,NaN
1,F1-2,21 March 2007,01:10,Marshall Islands,Falcon 1,DemoSat,NaN,NaN,NaN,DARPA,Government,United States,Failure,Engine Shutdown During Launch,NaN,NaN
2,F1-3,3 August 2008,03:34,Marshall Islands,Falcon 1,Trailblazer,Communication Satellite,NaN,NaN,ORS,Government,United States,Failure,Collision During Launch,NaN,NaN
3,F1-3,3 August 2008,03:34,Marshall Islands,Falcon 1,"PRESat, NanoSail-D",Research Satellites,8.0,NaN,NASA,Government,United States,Failure,Collision During Launch,NaN,NaN
4,F1-3,3 August 2008,03:34,Marshall Islands,Falcon 1,Explorers,Human Remains,NaN,NaN,Celestis,Business,United States,Failure,Collision During Launch,NaN,NaN


In [11]:
#Format Dates to MM/DD/YYYY
spacexdf['Launch Date'] = spacexdf['Launch Date'].map(lambda x: dateutil.parser.parse(x))\
                                                    .map(lambda x: x.strftime('%Y-%m-%d'))

In [12]:
#rename columns prior to inserting into MYSQL
spacexdf = spacexdf.rename(index=str, columns={"Flight Number": "flight_number", 
                                            "Launch Date": "launch_date",
                                            "Launch Time": "launch_time",
                                            "Launch Site": "launch_site",
                                            "Vehicle Type": "vehicle_type",
                                            "Payload Name": "payload_name",
                                            "Payload Type": "payload_type",
                                            "Payload Mass (kg)": "payload_mass_kg",
                                            "Payload Orbit": "payload_orbit",
                                            "Customer Name": "customer_name",
                                            "Customer Type": "customer_type",
                                            "Customer Country": "customer_country",
                                            "Mission Outcome": "mission_outcome",
                                            "Failure Reason": "failure_reason",
                                            "Landing Type": "landing_type",
                                            "Landing Outcome": "landing_outcome" })

In [13]:
spacexdf.head()

,flight_number,launch_date,launch_time,launch_site,vehicle_type,payload_name,payload_type,payload_mass_kg,payload_orbit,customer_name,customer_type,customer_country,mission_outcome,failure_reason,landing_type,landing_outcome
0,F1-1,2006-03-24,22:30,Marshall Islands,Falcon 1,FalconSAT-2,Research Satellite,19.5,NaN,DARPA,Government,United States,Failure,Engine Fire During Launch,NaN,NaN
1,F1-2,2007-03-21,01:10,Marshall Islands,Falcon 1,DemoSat,NaN,NaN,NaN,DARPA,Government,United States,Failure,Engine Shutdown During Launch,NaN,NaN
2,F1-3,2008-08-03,03:34,Marshall Islands,Falcon 1,Trailblazer,Communication Satellite,NaN,NaN,ORS,Government,United States,Failure,Collision During Launch,NaN,NaN
3,F1-3,2008-08-03,03:34,Marshall Islands,Falcon 1,"PRESat, NanoSail-D",Research Satellites,8.0,NaN,NASA,Government,United States,Failure,Collision During Launch,NaN,NaN
4,F1-3,2008-08-03,03:34,Marshall Islands,Falcon 1,Explorers,Human Remains,NaN,NaN,Celestis,Business,United States,Failure,Collision During Launch,NaN,NaN


In [14]:
#Grouped by customer country and counting the launch dates
Spacex_date_country = spacexdf.groupby(["customer_country"])["launch_date"].count()
Spacex_date_country

customer_country
Bermuda             2
Canada              1
China               2
France (Mexico)     2
Israel              1
Japan               2
Luxembourg          2
Malaysia            1
Thailand            2
Turkmenistan        1
United States      23
Name: launch_date, dtype: int64

# Nasa Facilities

In [15]:
nasadf.head()

,Center,Center Search Status,Facility,FacilityURL,Occupied,Status,URL Link,Record Date,Last Update,Country,Location,City,State,Zipcode
0,Stennis Space Center,Public,Test Stand A-2 #4122,NaN,01/01/1964 12:00:00 AM,Inactive,http://rockettest.nasa.gov,03/01/1996 12:00:00 AM,04/06/2015 12:00:00 AM,US,"39529-6000 (30.385948, -89.604486)",Stennis Space Center,MS,39529-6000
1,Ames Research Center,Public,N229 - EXPER. AEROTHERMODYNAMIC FAC.: ELECTRIC...,NaN,01/01/1961 12:00:00 AM,Active,http://thermo-physics.arc.nasa.gov,03/01/1996 12:00:00 AM,06/13/2014 12:00:00 AM,US,"94035 (37.41412, -122.052585)",Moffett Field,CA,94035
2,Goddard Space Flight Center,Public,"500 WSC: Antenna, 10m S-band",NaN,01/01/1977 12:00:00 AM,Active,NaN,03/01/1996 12:00:00 AM,08/15/2013 12:00:00 AM,US,"20771 (38.99538, -76.853161)",Greenbelt,MD,20771
3,Glenn Research Center,Public,Vacuum Facility 11 (VF-11) Bldg. 16,NaN,NaN,Active,http://facilities.grc.nasa.gov/epl/index.html,07/15/2010 12:00:00 AM,03/04/2015 12:00:00 AM,US,"44135 (41.430364, -81.808561)",Cleveland,OH,44135
4,Marshall Space Flight Center,Public,Materials Testing & Research Facility 4464,NaN,NaN,Active,NaN,03/08/2007 12:00:00 AM,03/31/2014 12:00:00 AM,US,"35812 (34.729538, -86.585283)",Huntsville,AL,35812


In [16]:
#Extract Lat and Long from location column into separate columns
latlong = nasadf['Location'].str.strip('()')\
                   .str.split(', ', expand=True)\
                   .rename(columns={0:'Latitude', 1:'Longitude'})

#Remove Zip code and punctuation from location column
latlong['Latitude'] = latlong['Latitude'].map(lambda x: x[-9:])\
                        .str.replace('(','')

#Add lat/long to df
nasadf = pd.merge(nasadf, latlong, left_index=True, right_index=True)

#Remove time stamp from occupied, record date and last update columns
nasadf['Record Date'] = nasadf['Record Date'].map(lambda x: dateutil.parser.parse(x))\
                                                .map(lambda x: x.strftime('%Y-%m-%d'))

nasadf['Occupied'] = nasadf['Occupied'].map(lambda x: dateutil.parser.parse(x) if pd.notnull(x) else np.nan)\
                                        .map(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else np.nan)

nasadf['Last Update'] = nasadf['Last Update'].map(lambda x: dateutil.parser.parse(x)if pd.notnull(x) else np.nan)\
                                            .map(lambda x: x.strftime('%Y-%m-%d')if pd.notnull(x) else np.nan)
    
#Combine City and State into location column
nasadf['Location'] = nasadf['City'] + ', ' + nasadf['State']

# Drop unneeded columns
nasadf.drop(['FacilityURL'],axis=1,inplace=True)

In [17]:
#rename columns prior to inserting into MYSQL
nasadf = nasadf.rename(index=str, columns={"Center": "center", 
                                            "Center Search Status": "center_search_status",
                                            "Facility": "facility",
                                            "Occupied": "occupied",
                                            "Status": "status",
                                            "URL Link": "url",
                                            "Record Date": "record_date",
                                            "Last Update": "last_update",
                                            "Country": "country",
                                            "Location": "location",
                                            "City": "city",
                                            "State": "state",
                                            "Zipcode": "zipcode",
                                            "Latitude": "latitude",
                                            "Longitude": "longitude" })

In [18]:
nasadf.head()

,center,center_search_status,facility,occupied,status,url,record_date,last_update,country,location,city,state,zipcode,latitude,longitude
0,Stennis Space Center,Public,Test Stand A-2 #4122,1964-01-01,Inactive,http://rockettest.nasa.gov,1996-03-01,2015-04-06,US,"Stennis Space Center, MS",Stennis Space Center,MS,39529-6000,30.385948,-89.604486
1,Ames Research Center,Public,N229 - EXPER. AEROTHERMODYNAMIC FAC.: ELECTRIC...,1961-01-01,Active,http://thermo-physics.arc.nasa.gov,1996-03-01,2014-06-13,US,"Moffett Field, CA",Moffett Field,CA,94035,37.41412,-122.052585
2,Goddard Space Flight Center,Public,"500 WSC: Antenna, 10m S-band",1977-01-01,Active,NaN,1996-03-01,2013-08-15,US,"Greenbelt, MD",Greenbelt,MD,20771,38.99538,-76.853161
3,Glenn Research Center,Public,Vacuum Facility 11 (VF-11) Bldg. 16,NaN,Active,http://facilities.grc.nasa.gov/epl/index.html,2010-07-15,2015-03-04,US,"Cleveland, OH",Cleveland,OH,44135,41.430364,-81.808561
4,Marshall Space Flight Center,Public,Materials Testing & Research Facility 4464,NaN,Active,NaN,2007-03-08,2014-03-31,US,"Huntsville, AL",Huntsville,AL,35812,34.729538,-86.585283


In [19]:
#Grouped by vehicle type and payload mass
nasa_date_countries = nasadf.groupby(["center","city"])["occupied"].count()
nasa_date_countries

center                             city                
Ames Research Center               Moffett Field            12
Armstrong Flight Research Center   Edwards                   3
Glenn Research Center              Cleveland                32
Goddard Space Flight Center        Greenbelt                24
Jet Propulsion Lab                 PASADENA                 34
Johnson Space Center               Houston                   3
Kennedy Space Center               Kennedy Space Center     83
Langley Research Center            Hampton                  42
Marshall Space Flight Center       Huntsville              106
Michoud Assembly Facility          New Orleans              16
NASA Aircraft Management Division  Washington                0
Stennis Space Center               Stennis Space Center     28
Wallops Flight Facility/GSFC       Wallops Island           10
Name: occupied, dtype: int64

# UCS Satellites

In [20]:
UCSSatellite_db.columns

Index(['Name of Satellite, Alternate Names', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments',
       'Source Used for Orbital Data', 'Source'],
      dtype='object')

In [21]:
UCSSatellite_db['Date of Launch'] = UCSSatellite_db['Date of Launch'].map(lambda x: dateutil.parser.parse(x) if pd.notnull(x) else np.nan)
UCSSatellite_db['Date of Launch'] = UCSSatellite_db['Date of Launch'].map(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else np.nan)

In [22]:
UCSSatellite_db = UCSSatellite_db.rename(index=str, columns={"Name of Satellite, Alternate Names":"name_of_satellite", 
                                    "Country/Org of UN Registry":"un_registry",
                                    "Country of Operator/Owner":"country",
                                    "Operator/Owner": "operator",
                                    "Users": "users",
                                    "Purpose": "purpose",
                                    "Detailed Purpose": "detailed_purpose",
                                    "Class of Orbit": "class_of_orbit",
                                    "Type of Orbit": "type_of_orbit",
                                    "Longitude of GEO (degrees)": "geo_longitude",
                                    "Perigee (km)":"perigee",
                                    "Apogee (km)":"apogee",
                                    "Eccentricity": "eccentricity",
                                    "Inclination (degrees)":"inclination",
                                    "Period (minutes)":"period",
                                    "Launch Mass (kg.)":"launch_mass",
                                    "Dry Mass (kg.)":"dry_mass",
                                    "Power (watts)":"Power",
                                    "Date of Launch":"launch_date",
                                    "Expected Lifetime (yrs.)":"expected_lifetime",
                                    "Contractor":"contractor",
                                    "Country of Contractor":"country_of_contractor",
                                    "Launch Site":"launch_site",
                                    "Launch Vehicle":"launch_vehicle",
                                    "COSPAR Number":"cospar_number",
                                    "NORAD Number":"norad_number",
                                    "Comments":"comments",
                                    "Source Used for Orbital Data":"orbital_data_source",
                                    "Source":"source"
                                    })
UCSSatellite_db.columns

Index(['name_of_satellite', 'un_registry', 'country', 'operator', 'users',
       'purpose', 'detailed_purpose', 'class_of_orbit', 'type_of_orbit',
       'geo_longitude', 'perigee', 'apogee', 'eccentricity', 'inclination',
       'period', 'launch_mass', 'dry_mass', 'Power', 'launch_date',
       'expected_lifetime', 'contractor', 'country_of_contractor',
       'launch_site', 'launch_vehicle', 'cospar_number', 'norad_number',
       'comments', 'orbital_data_source', 'source'],
      dtype='object')

In [23]:
#Comments column from float to string
UCSSatellite_db["comments"] = UCSSatellite_db["comments"].astype('str')
UCSSatellite_db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1957 entries, 0 to 1956
Data columns (total 29 columns):
name_of_satellite        1957 non-null object
un_registry              1957 non-null object
country                  1957 non-null object
operator                 1957 non-null object
users                    1957 non-null object
purpose                  1957 non-null object
detailed_purpose         718 non-null object
class_of_orbit           1957 non-null object
type_of_orbit            1348 non-null object
geo_longitude            1956 non-null float64
perigee                  1957 non-null object
apogee                   1957 non-null object
eccentricity             1957 non-null float64
inclination              1957 non-null float64
period                   1957 non-null object
launch_mass              1813 non-null object
dry_mass                 484 non-null float64
Power                    640 non-null object
launch_date              1957 non-null object
expected_lifetime      

In [24]:
#Remove illegal characters from numerical columns
UCSSatellite_db["launch_mass"] = UCSSatellite_db["launch_mass"]\
                                .apply(lambda x: x.replace(',','') if pd.notnull(x) else np.nan)
UCSSatellite_db["perigee"] = UCSSatellite_db["perigee"]\
                            .apply(lambda x: x.replace(',','') if pd.notnull(x) else np.nan)

UCSSatellite_db["apogee"] = UCSSatellite_db["apogee"]\
                            .apply(lambda x: x.replace(',','') if pd.notnull(x) else np.nan)

UCSSatellite_db["Power"] = UCSSatellite_db["Power"]\
                            .apply(lambda x: x.replace(',','') if pd.notnull(x) else np.nan)
UCSSatellite_db["Power"] = UCSSatellite_db["Power"]\
                            .apply(lambda x: x.replace(' (EOL)','') if pd.notnull(x) else np.nan)
UCSSatellite_db["Power"] = UCSSatellite_db["Power"]\
                            .apply(lambda x: x.replace(' (BOL)','') if pd.notnull(x) else np.nan)

UCSSatellite_db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1957 entries, 0 to 1956
Data columns (total 29 columns):
name_of_satellite        1957 non-null object
un_registry              1957 non-null object
country                  1957 non-null object
operator                 1957 non-null object
users                    1957 non-null object
purpose                  1957 non-null object
detailed_purpose         718 non-null object
class_of_orbit           1957 non-null object
type_of_orbit            1348 non-null object
geo_longitude            1956 non-null float64
perigee                  1957 non-null object
apogee                   1957 non-null object
eccentricity             1957 non-null float64
inclination              1957 non-null float64
period                   1957 non-null object
launch_mass              1813 non-null object
dry_mass                 484 non-null float64
Power                    640 non-null object
launch_date              1957 non-null object
expected_lifetime      

In [25]:
UCSSatellite_db.loc[UCSSatellite_db["perigee"].notnull(), "perigee"] = UCSSatellite_db.loc\
                                    [UCSSatellite_db["perigee"].notnull(), "perigee"].astype(int)

UCSSatellite_db.loc[UCSSatellite_db["apogee"].notnull(), "apogee"] = UCSSatellite_db.loc\
                                    [UCSSatellite_db["apogee"].notnull(), "apogee"].astype(int)

UCSSatellite_db.loc[UCSSatellite_db["launch_mass"].notnull(), "launch_mass"] = UCSSatellite_db.loc\
                                    [UCSSatellite_db["launch_mass"].notnull(), "launch_mass"].astype(int)

UCSSatellite_db.loc[UCSSatellite_db["Power"].notnull(), "Power"] = UCSSatellite_db.loc\
                                    [UCSSatellite_db["Power"].notnull(), "Power"].astype(float)

In [26]:
UCSSatellite_db.head()

,name_of_satellite,un_registry,country,operator,users,purpose,detailed_purpose,class_of_orbit,type_of_orbit,geo_longitude,...,expected_lifetime,contractor,country_of_contractor,launch_site,launch_vehicle,cospar_number,norad_number,comments,orbital_data_source,source
0,3Cat-1,NR (11/18),Spain,Universitat Politècnica de Catalunya,Civil,Technology Development,NaN,LEO,NaN,0.0,...,NaN,Universitat Politècnica de Catalunya,Spain,Satish Dhawan Space Centre,PSLV,2018-096,43723,Student built.,Estimated,https://spaceflightnow.com/2018/11/29/indian-r...
1,BlackSky Global 1,NR (11/18),USA,BlackSky Global,Commercial,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,0.0,...,NaN,Spaceflight Industries,USA,Satish Dhawan Space Centre,PSLV,2018-096B,43720,Follow-up to pathfinder.,JM/11_18,https://spaceflightnow.com/2018/11/29/indian-r...
2,Centauri-1,NR (11/18),Australia,Fleet Space Technologies,Commercial,Communications,NaN,LEO,Sun-Synchronous,0.0,...,NaN,Fleet Space Technologies,Australia,Satish Dhawan Space Centre,PSLV,2018-096,43723,Test cubesat.,Estimated,https://spaceflightnow.com/2018/11/29/indian-r...
3,Cicero-8 (Community Initiative for Cellular Ea...,NR (11/18),USA,GeoOptics Inc.,Commercial,Earth Observation,Meteorology,LEO,Sun-Synchronous,0.0,...,NaN,Tyvak Nanosatellite Systems,USA,Satish Dhawan Space Centre,PSLV,2018-096C,73721,nan,JM/11_18,https://spaceflightnow.com/2018/11/29/indian-r...
4,Dove 3r-10 (0 Flock 3R-10 1059),NR (11/18),USA,"Planet Labs, Inc.",Commercial,Earth Observation,Optical Imaging,LEO,Sun-Synchronous,0.0,...,NaN,"Planet Labs, Inc.",USA,Satish Dhawan Space Centre,PSLV,2018-096E,43723,nan,JM/11_18,https://spaceflightnow.com/2018/11/29/indian-r...


In [27]:
#Grouped by users and counting by countries (this would show which use is more popular in each country)
ucs_date_countries = UCSSatellite_db.groupby(["users"])["country"].count()
ucs_date_countries

users
Civil                             145
Civil/Government                    2
Commercial                        845
Commercial                          1
Commercial/Military                 2
Government                        385
Government/Civil                   28
Government/Commercial             121
Government/Commercial/Military      1
Government/Military                 5
Military                          301
Military                            1
Military/Civil                      3
Military/Commercial                72
Military/Government                45
Name: country, dtype: int64

# Load

In [28]:
#Find table details for SQL Sche,a
for column in spacex:
    thislist = []
    try:
        for row in spacex[column]:

            thislist.append(len(row))
            
        print(f"{column} {max(thislist)}")
        if max(thislist) ==min(thislist):
            print("not variable\n---")
        else:
            print("variable\n---")
    except:
        print(f"{column} len not found\n---")

launch_site 56
variable
---
launch_success len not found
---
mission_name 39
variable
---
launch_date 10
not variable
---
rocket_reused len not found
---
rocket_name 12
variable
---
rocket_type 8
variable
---
failure_reason len not found
---
customer 34
variable
---
customer_country 14
variable
---
orbit 5
variable
---
payload_id 31
variable
---
payload_mass_kg len not found
---
payload_type 18
variable
---


In [29]:
for column in UCSSatellite_db:
    thislist = []
    try:
        for row in UCSSatellite_db[column]:

            thislist.append(len(row))
            
        print(f"{column} {max(thislist)}")
        if max(thislist) ==min(thislist):
            print("not variable\n---")
        else:
            print("variable\n---")
    except:
        print(f"{column} len not found\n---")
        

name_of_satellite 147
variable
---
un_registry 20
variable
---
country 33
variable
---
operator 143
variable
---
users 30
variable
---
purpose 46
variable
---
detailed_purpose len not found
---
class_of_orbit 10
variable
---
type_of_orbit len not found
---
geo_longitude len not found
---
perigee len not found
---
apogee len not found
---
eccentricity len not found
---
inclination len not found
---
period 8
variable
---
launch_mass len not found
---
dry_mass len not found
---
Power len not found
---
launch_date 10
not variable
---
expected_lifetime len not found
---
contractor 126
variable
---
country_of_contractor 29
variable
---
launch_site 36
variable
---
launch_vehicle 29
variable
---
cospar_number 11
variable
---
norad_number len not found
---
comments 400
variable
---
orbital_data_source len not found
---
source len not found
---


In [30]:
# Connect to local database
from config import rds_connection_string

engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [31]:
launchlog.to_sql(name='launch_log', con=engine, if_exists='append', index=False)
nasadf.to_sql(name='nasa_facilities', con=engine, if_exists='append', index=False)
spacex.to_sql(name='space_x_launches', con=engine, if_exists='append', index=False)
UCSSatellite_db.to_sql(name='ucs_satellites', con=engine, if_exists='append', index=False)

In [32]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from launch_log', con=engine).head()

,launch,launch_date_utc,COSPAR,pl_name,orig_pl_name,SATCAT,lv_type,lv_sn,site,suc,Ref,launch_date
0,1957 ALP,1957 Oct 4 1928:34,1957 ALP 2,1-y ISZ,PS-1,S00002,Sputnik 8K71PS,M1-PS,NIIP-5 LC1,S,Energiya,1957-10-04
1,1957 BET,1957 Nov 3 0230:42,1957 BET 1,2-y ISZ,PS-2,S00003,Sputnik 8K71PS,M1-2PS,NIIP-5 LC1,S,Grahn-WWW,1957-11-03
2,1958 ALP,1958 Feb 1 0347:56,1958 ALP,Explorer 1,Explorer 1,S00004,Jupiter C,RS-29 UE,CC LC26A,S,JunoFam,1958-02-01
3,1958 BET,1958 Mar 17 1215:41,1958 BET 2,Vanguard I,Vanguard Test Satellite,S00005,Vanguard,TV-4,CC LC18A,S,SP-4202,1958-03-01
4,1958 GAM,1958 Mar 26 1738:03,1958 GAM,Explorer 3,Explorer 3,S00006,Jupiter C,RS-24 UT,CC LC5,S,JunoFam,1958-03-02


In [33]:
engine.table_names()

['launch_log', 'nasa_facilities', 'space_x_launches', 'ucs_satellites']

# Query

In [34]:
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')
session = Session(engine)
inspector = inspect(engine)
inspector.get_table_names()

['launch_log', 'nasa_facilities', 'space_x_launches', 'ucs_satellites']

In [35]:
cities = pd.read_sql_query('select location from nasa_facilities', index_col=None,con=engine)
cities['location'].to_list()
launch_log = Table('launch_log', MetaData(), autoload=True, autoload_with=engine)
type(launch_log)

sqlalchemy.sql.schema.Table

In [36]:
launchlog.describe()

,launch,launch_date_utc,COSPAR,pl_name,orig_pl_name,SATCAT,lv_type,lv_sn,site,suc,Ref,launch_date
count,8641,8641,8641,8641,8641,8641,5464,5464,5464,5464,5464,8641
unique,5464,5464,8614,8260,6852,8641,345,5171,204,2,729,2489
top,2017-008,2017 Feb 15 0358,1998-069F,Molniya-1,Strela-1M,S05211,Soyuz-U,-,NIIP-53 LC41/1,S,AK15/RAEX,2017-02-01
freq,104,104,3,85,360,1,615,71,303,5385,1257,107


In [37]:
nasadf.describe()

,center,center_search_status,facility,occupied,status,url,record_date,last_update,country,location,city,state,zipcode,latitude,longitude
count,485,485,485,393,399,145,485,479,485,485,485,485,485,485,485
unique,13,1,485,58,4,62,55,34,1,13,13,10,13,13,13
top,Marshall Space Flight Center,Public,Mobile Launcher Platform 1,1966-01-01,Active,http://rockettest.nasa.gov,1996-03-01,2015-06-22,US,"Huntsville, AL",Huntsville,AL,35812,34.729538,-86.585283
freq,136,485,1,42,371,24,354,84,485,136,136,136,136,136,136


In [38]:
spacex.describe()

,payload_mass_kg
count,72.000000
mean,3950.345139
std,2815.771273
min,20.000000
25%,2162.250000
50%,3247.500000
75%,5320.962500
max,12259.000000


In [39]:
UCSSatellite_db.describe()

,geo_longitude,perigee,apogee,eccentricity,inclination,dry_mass,expected_lifetime,norad_number
count,1956.000000,1957.000000,1957.000000,1957.000000,1957.000000,484.000000,1125.000000,1957.000000
mean,5.930486,11965.081247,13551.871231,0.016845,58.525692,1274.126033,9.811778,37815.624936
std,50.414674,15835.926229,21213.555971,0.103708,41.540139,1205.794610,4.966454,6086.240277
min,-179.800000,187.000000,36.000000,-0.736000,0.000000,1.000000,0.250000,7530.000000
25%,0.000000,500.000000,544.000000,0.000261,2.100000,556.000000,5.000000,34264.000000
50%,0.000000,710.000000,786.000000,0.000801,64.820000,889.500000,10.000000,40082.000000
75%,0.000000,35732.000000,35791.000000,0.001450,97.500000,1814.000000,15.000000,42711.000000
max,328.500000,62200.000000,330000.000000,0.957000,143.400000,10000.000000,30.000000,73721.000000
